In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip
!rm *.zip

In [2]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
# import csv data into dataframes
df_books = pd.read_csv(
	books_filename,
	encoding = "ISO-8859-1",
	sep=";",
	header=0,
	names=['isbn', 'title', 'author'],
	usecols=['isbn', 'title', 'author'],
	dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
	ratings_filename,
	encoding = "ISO-8859-1",
	sep=";",
	header=0,
	names=['user', 'isbn', 'rating'],
	usecols=['user', 'isbn', 'rating'],
	dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [3]:
#Merging ratings with books to simplify work
df = pd.merge(df_ratings, df_books, on='isbn')
df = df.drop(columns=['author'])#not useing authors so lets remvoe them

#remove users w/less than 200 ratings & books w/less than 100 ratings
user_count = df_ratings['user'].value_counts()
book_count = df_ratings['isbn'].value_counts()
user_mask = user_count[user_count >= 200].index
book_mask = book_count[book_count >= 100].index
valid_mask = (df['user'].isin(user_mask)) & (df['isbn'].isin(book_mask))
df = df[valid_mask]

In [8]:
#create matrix for training
table = df.pivot_table(index='title', columns='user', values='rating').fillna(0)
matrix = csr_matrix(table.values)
model = NearestNeighbors(n_neighbors=5)
model.fit(matrix)

NearestNeighbors()

In [12]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
	index = table.index.get_loc(book)

	distances, indices = model.kneighbors(table.iloc[index,:].values.reshape(1,-1),n_neighbors=6)
	distances = distances.flatten()
	indices = indices.flatten()
	
	recommended_books = []#[book,distance]
	for i in range(1, len(distances)):
		recommended_books.append([table.index[indices[i]], distances[i]])
	recommended_books = [book,recommended_books]
	return recommended_books

In [13]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Pleading Guilty', 35.29872512817383],
  ['Long After Midnight', 35.74912643432617],
  ['Jackdaws', 36.09709167480469],
  ['Eaters of the Dead', 36.33180236816406],
  ['Last Man Standing', 36.701499938964844]]]

In [14]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
	test_pass = True
	recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
	if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
		test_pass = False
	recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
	recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
	for i in range(2): 
		if recommends[1][i][0] not in recommended_books:
			test_pass = False
		if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
			test_pass = False
	if test_pass:
		print("You passed the challenge! 🎉🎉🎉🎉🎉")
	else:
		print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Surgeon', 61.28621292114258], ['Unspeakable', 61.52235412597656], ['The Perks of Being a Wallflower', 61.57921600341797], ['Gap Creek: The Story Of A Marriage', 61.67657470703125], ['The Weight of Water', 61.757591247558594]]]
You haven't passed yet. Keep trying!
